GEODATA-HARVESTER WITH SETTINGS WIDGET
--------------------------------------

The Geodata-Harvester enables researchers with reusable workflows for automatic data extraction from a range of data sources including spatial-temporal processing into useable formats. User provided data is auto-completed with a suitable set of spatial- and temporal-aligned covariates as a ready-made dataset for machine learning and agriculture models. In addition, all requested data layer maps are automatically extracted and aligned for a specific region and time period.

The main workflow of the Harvester is as follows:

Options and user settings (e.g., data layer selections, spatial coverage, temporal constraints, i/o directory names) are defined by the user in the notebook settings menu or can be loaded with a settings yaml file (e.g., settings/settings_test). All settings are also saved in a yaml file for reusability.

The notebook imports settings and all Python modules that include functionality to download and extract data for each data source. After settings are read in, checked, and processed into valid data retrieval (API) queries, all selected data layers are sequentially downloaded and then processed into a clean dataframe table and co-registered raster maps. The entire workflow can be run either completely automatically or individually by selecting only certain process parts in the Notebook.
Additional data sources can be best added by writing the API handlers and extraction functionalities as separate Python module, which are then imported by the Notebook. Currently the following data sources are supported by the following modules:

- 'getdata_slga.py': Soil Data from Soil and Landscape Grid of Australia (SLGA)
- 'getdata_landscape': Landscape data from Soil and Landscape Grid of Australia (SLGA)
- 'getdata_silo.py': Climate Data from SILO
- 'getdata_dem.py: 'National Digital Elevation Model (DEM) 1 Second plus Slope and Apect calculation
- 'getdata_dea_nci.py: 'Digital Earth Australia's (DEA) Geoscience Earth Observations via NCI server
- 'getdata_dea.py: 'Digital Earth Australia's (DEA) Geoscience Earth Observations via Open Web Service server provided by DEA
- 'getdata_radiometric.py': Geoscience Australia National Geophysical Compilation Sub-collection Radiometrics
'eeharvest': Google Earth Engine API integration handler
For more details. please see README and the Data Overview page.

This notebook is part of the Data Harvester project developed for the Agricultural Research Federation (AgReFed).

Copyright 2023 Sydney Informatics Hub (SIH), The University of Sydney

### Import libraries

In [1]:
import os
import time
from datetime import datetime
from os.path import exists
from pathlib import Path
from types import SimpleNamespace

# Import harvest function from geodata_harvester
from geodata_harvester import harvest
# Import widget library
from geodata_harvester.widgets import harvesterwidgets as hw

### Settings via interactive widget

Set settings such as data-layer names, region, and dates in widget window. The widget window might take a few seconds to open in the notebook.

In [2]:
tab_nest, w_settings, names_settings, w_load = hw.gen_maintab()
#Note: the display screen may take a couple of seconds more after loading
time.sleep(2)
display(tab_nest) 

### Evaluate and save settings as YAML file

In [7]:
if w_load.value == None:
    dict_settings = hw.eval_widgets(w_settings, names_settings)
    # Convert settings from dictionary to SimpleNamespace (so all settings names available as settings.xxxname)
    settings = SimpleNamespace(**dict_settings)
    # Check if output path exists, if not create it:
    os.makedirs(settings.outpath, exist_ok=True) 
    # Save settings to yaml file:
    fname_settings = os.path.join(settings.outpath, 'settings_saved.yaml')
    hw.save_dict_settings(dict_settings, fname_settings)
else:
    print(f'Settings loaded from {w_load.value}')
    settings = hw.load_settings(w_load.value)
# Print settings
hw.print_settings(settings)

Settings saved to file test_widgets/settings_saved.yaml
Settings loaded:
----------------
settings.infile : /Users/seb/CTDS/Projects/AgReFed/Harvester/geodata-harvester/notebooks/testdata/example-site_llara.csv
settings.outpath : test_widgets
settings.colname_lng : Long
settings.colname_lat : Lat
settings.target_bbox : [149.769345, -30.335861, 149.949173, -30.206271]
settings.target_res : 6.000000000000001
settings.date_min : 2023-01-01
settings.temp_intervals : 2
settings.date_max : 2023-01-10
settings.temp_buffer : 1
settings.target_sources:
   'SLGA': {'Bulk_Density': ['0-5cm']}
   'SILO': {'monthly_rain': ['median']}
   'DEA': ['s2_barest_earth']
   'DEM': ['DEM']
   'Radiometric': ['radmap2019_grid_dose_terr_awags_rad_2019']
   'Landscape': ['Slope']
   'GEE': {'preprocess': {'collection': 'LANDSAT/LC08/C02/T1_L2', 'spectral': 'NDVI', 'reduce': 'median', 'mask_clouds': True, 'mask_probability': None}, 'download': {'bands': ['7', '8']}}


### Harvest

The harvest function executes automatically all download and processing steps for all data layers from the web as specified in settings file above.
Filenames of the processed files and some metadata is saved in a dataframe df.

In [11]:
df = harvest.run(fname_settings, return_df = True)

Starting the data harvester -----
ℹ Found the following 7 sources: ['DEA', 'DEM', 'GEE', 'Landscape', 'Radiometric', 'SILO', 'SLGA']

⚑ Earth Engine API already authenticated

⌛ Downloading Google Earth Engine data...
✔ YAML schema validated
Running preprocess() -----
⊙ Applying scale, offset and cloud masks... 2.1s                                                               
⊙ Calculating spectral indices: NDVI... 0.9s                                                                   
⊙ Reducing image pixels by median 0.0s                                                                         
✔ Preprocessing complete
Running download() -----
ℹ Band(s) selected: NDVI
ℹ Setting scale to ~160.2m, converted from 6.000000000000001 arcsec at latitude -30.27
ℹ Setting download dir to test_widgets
⊙ Downloading ee_LANDSAT_e3314e4b.tif (!) 0.7s 


EEException: Image.select: Pattern 'NDVI' did not match any bands.

### Inspect result dataframe

In [4]:
# Inspect either entire generated dataframe with 
# df
# or only the first rows with
df.head()

,Longitude,Latitude,geometry,ee_LANDSAT_bfafc43c_median,landsat_barest_earth,ga_ls_ard_3_2022-9-30,ga_ls_ard_3_2022-10-1,ga_ls_ard_3_2022-10-2,ga_ls_ard_3_2022-10-3,ga_ls_ard_3_2022-10-4,...,DEM,landscape_Slope,landscape_Aspect,landscape_Relief_300m,daily_rain_mean,max_temp_median,min_temp_median,monthly_rain_sum,Bulk_Density_0-5cm,Clay_0-5cm
0,149.852680,-30.264663,POINT (149.85268 -30.26466),0.654711,1059,-999,-999,-999,-999,-999,...,244.658585,1.046624,209.138062,10.463379,0.000000,22.700001,9.8,189.500000,1.368779,27.214527
1,149.884838,-30.265302,POINT (149.88484 -30.26530),0.415696,1082,-999,-999,-999,-999,-999,...,264.428772,1.001000,279.542847,6.037811,0.199951,22.600000,9.5,189.000000,1.362662,31.956041
2,149.884838,-30.265302,POINT (149.88484 -30.26530),0.415696,1082,-999,-999,-999,-999,-999,...,264.428772,1.001000,279.542847,6.037811,0.199951,22.600000,9.5,189.000000,1.362662,31.956041
3,149.838791,-30.278542,POINT (149.83879 -30.27854),0.594925,1092,-999,-999,-999,-999,-999,...,233.005081,0.841430,242.743683,4.798782,0.000000,22.900000,10.1,173.199951,1.360451,32.675858
4,149.830843,-30.275437,POINT (149.83084 -30.27544),0.708653,1160,-999,-999,-999,-999,-999,...,230.575439,1.062537,242.921112,5.204880,0.000000,22.900000,10.1,173.199951,1.334362,35.097813
